# Introduction to the HugeCTR Embedding Collection

## About the HugeCTR embedding collection

This notebook demonstrates the following:

* Introduce the API of the embedding collection.
* Introduce the embedding table placement strategy (ETPS) and how to configure ETPS in embedding collection.
* Show how to use an embedding collection in a DLRM model with the Criteo dataset for training and evaluation.
The notebook shows two different ETPS as reference.

## Concepts and API Reference

The following key classes and configuration file are used in this notebook:

* `hugectr.EmbeddingTableConfig`
* `hugectr.EmbeddingPlanner`
* JSON plan file for the ETPS.

For the concepts and API reference information about the classes and file, see the [Overview of Using the HugeCTR Embedding Collection](https://nvidia-merlin.github.io/HugeCTR/master/hugectr_layer_book.html#overview-of-using-the-hugectr-embedding-collection) in the HugeCTR Layer Classes and Methods information.

## Use an Embedding Colleciton with a DLRM Model

### Prepare the Data

You can follow the instruction under [samples/deepfm/README.md#Preprocess the Dataset Through NVTabular](../samples/deepfm/README.md#Preprocess_the_Dataset_Through_NVTabular) to prepare data.

### Prepare the Training Script

This notebook was developed with on single DGX-1 to run the DLRM model in this notebook. The GPU info in DGX-1 is as follows. It consists of 8 V100-SXM2 GPUs.

In [6]:
! nvidia-smi

Thu Jun 23 00:14:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   33C    P0    42W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

We build our train script through embedding collection API. And we will use command argument to pass plan file and test different ETPS.

In [25]:
%%writefile dlrm_train.py
import os
import sys
import hugectr

plan_file = sys.argv[1]
table_size_array = [203931, 18598, 14092, 7012, 18977, 4, 6385, 1245, 49, 186213, 71328, 67288, 11, 2168, 7338, 61, 4, 932, 15, 204515, 141526, 199433, 60919, 9137, 71, 34]

solver = hugectr.CreateSolver(max_eval_batches = 70,
                              batchsize_eval = 65536,
                              batchsize = 65536,
                              lr = 0.5,
                              warmup_steps = 300,
                              vvgpu = [[0,1,2,3,4,5,6,7]],
                              repeat_dataset = True,
                              i64_input_key = True,
                              metrics_spec = {hugectr.MetricsType.AverageLoss:0.0},
                              use_embedding_collection = True)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Parquet,
                                  source = ["./deepfm_data_nvt/train/_file_list.txt"],
                                  eval_source = "./deepfm_data_nvt/val/_file_list.txt",
                                  check_type=hugectr.Check_t.Non,
                                  slot_size_array = table_size_array)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.SGD,
                                    update_type = hugectr.Update_t.Local,
                                    atomic_update = True)
model = hugectr.Model(solver, reader, optimizer)

model.add(hugectr.Input(label_dim = 1, label_name = "label",
                        dense_dim = 13, dense_name = "dense",
                        data_reader_sparse_param_array = 
                        [hugectr.DataReaderSparseParam("data{}".format(i), 1, False, 1) for i in range(len(table_size_array))]))

# create embedding table
embedding_table_list = []
for i in range(len(table_size_array)):
    embedding_table_list.append(hugectr.EmbeddingTableConfig(table_id=i, 
                                                             max_vocabulary_size=table_size_array[i], 
                                                             ev_size=128, 
                                                             min_key=0, 
                                                             max_key=table_size_array[i]))
# create embedding planner and embedding collection
embedding_planner = hugectr.EmbeddingPlanner()
emb_vec_list = []
for i in range(len(table_size_array)):
    embedding_planner.embedding_lookup(table_config=embedding_table_list[i], 
                                       bottom_name=f"data{i}", 
                                       top_name=f"emb_vec{i}", 
                                       combiner="sum")
embedding_collection = embedding_planner.create_embedding_collection(plan_file)

model.add(embedding_collection)
# need concat
model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Concat,
                              bottom_names = ["emb_vec{}".format(i) for i in range(len(table_size_array))],
                              top_names = ["sparse_embedding1"],
                              axis = 1))

model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["dense"],
                            top_names = ["fc1"],
                            num_output=512))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc1"],
                            top_names = ["relu1"]))                           
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["relu1"],
                            top_names = ["fc2"],
                            num_output=256))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc2"],
                            top_names = ["relu2"]))                            
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["relu2"],
                            top_names = ["fc3"],
                            num_output=128))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc3"],
                            top_names = ["relu3"]))                              
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Interaction, # interaction only support 3-D input
                            bottom_names = ["relu3","sparse_embedding1"],
                            top_names = ["interaction1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["interaction1"],
                            top_names = ["fc4"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc4"],
                            top_names = ["relu4"]))                              
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["relu4"],
                            top_names = ["fc5"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc5"],
                            top_names = ["relu5"]))                              
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["relu5"],
                            top_names = ["fc6"],
                            num_output=512))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc6"],
                            top_names = ["relu6"]))                               
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["relu6"],
                            top_names = ["fc7"],
                            num_output=256))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc7"],
                            top_names = ["relu7"]))                                                                              
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["relu7"],
                            top_names = ["fc8"],
                            num_output=1))                                                                                           
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.BinaryCrossEntropyLoss,
                            bottom_names = ["fc8", "label"],
                            top_names = ["loss"]))
model.compile()
model.summary()
model.fit(max_iter = 1000, display = 100, eval_interval = 100, snapshot = 10000000, snapshot_prefix = "dlrm")


Overwriting dlrm_train.py


### ETPS: Data parallel + Localized
We want to put small size table as data parallel while for other tables, each table will be on single GPU and different GPU will hold different table(The same way we we use in `hugectr.LocalizedHashEmbedding`).

In [3]:
def print_plan(plan):
    for id,single_gpu_plan in enumerate(plan):
        print('single_gpu_plan index = {}'.format(id))
        for plan_attr in single_gpu_plan:
            for key in plan_attr:
                if (key != 'global_embedding_list'):
                    print('\t{}:{}'.format(key,plan_attr[key]))
                else:
                    prefix_len = len(key)
                    left_space_fill = ' '*prefix_len
                    print('\t{}:{}'.format(key,plan_attr[key][0]))
                    for index in range(1,len(plan_attr[key])):
                        print('\t{}:{}'.format(left_space_fill,plan_attr[key][index]))

def generate_plan(table_size_array, num_gpus, plan_file):
    
    mp_table = [i for i in range(len(table_size_array)) if table_size_array[i] > 6000]
    dp_table = [i for i in range(len(table_size_array)) if table_size_array[i] <= 6000]

    # place table across all gpus
    plan = []
    for gpu_id in range(num_gpus):
        single_gpu_plan = []
        mp_plan = {
          'local_embedding_list': [table_id for i, table_id in enumerate(mp_table) if i % num_gpus == gpu_id],
          'table_placement_strategy': 'mp'
        }
        dp_plan = {
          'local_embedding_list': dp_table,
          'table_placement_strategy': 'dp'
        }
        single_gpu_plan.append(mp_plan)
        single_gpu_plan.append(dp_plan)
        plan.append(single_gpu_plan)

    # generate global view of table placement
    mp_global_embedding_list = []
    dp_global_embedding_list = []
    for single_gpu_plan in plan:
        mp_global_embedding_list.append(single_gpu_plan[0]['local_embedding_list'])
        dp_global_embedding_list.append(single_gpu_plan[1]['local_embedding_list'])
    for single_gpu_plan in plan:
        single_gpu_plan[0]['global_embedding_list'] = mp_global_embedding_list
        single_gpu_plan[1]['global_embedding_list'] = dp_global_embedding_list
    print_plan(plan)
    # dump plan file
    import json
    with open(plan_file, 'w') as f:
        json.dump(plan, f, indent=4)

In [4]:
table_size_array = [203931, 18598, 14092, 7012, 18977, 4, 6385, 1245, 49, 186213, 71328, 67288, 11, 2168, 7338, 61, 4, 932, 15, 204515, 141526, 199433, 60919, 9137, 71, 34]
generate_plan(table_size_array, 8, "./dp_and_localized_plan.json")

single_gpu_plan index = 0
	local_embedding_list:[0, 11]
	table_placement_strategy:mp
	global_embedding_list:[0, 11]
	                     :[1, 14]
	                     :[2, 19]
	                     :[3, 20]
	                     :[4, 21]
	                     :[6, 22]
	                     :[9, 23]
	                     :[10]
	local_embedding_list:[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
	table_placement_strategy:dp
	global_embedding_list:[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
	                     :[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
	                     :[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
	                     :[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
	                     :[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
	                     :[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
	                     :[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
	                     :[5, 7, 8, 12, 13, 15, 16, 17, 18, 24, 25]
single_gpu_plan index = 1
	local_embedding_list:[1, 14]
	

In [5]:
!python3 dlrm_train.py ./dp_and_localized_plan.json

HugeCTR Version: 3.7
====================================================Model Init=====================================================
[HCTR][08:41:17.942][WARNING][RK0][main]: The model name is not specified when creating the solver.
[HCTR][08:41:17.942][INFO][RK0][main]: Global seed is 1323844045
[HCTR][08:41:18.400][INFO][RK0][main]: Device to NUMA mapping:
  GPU 0 ->  node 0
  GPU 1 ->  node 0
  GPU 2 ->  node 0
  GPU 3 ->  node 0
  GPU 4 ->  node 1
  GPU 5 ->  node 1
  GPU 6 ->  node 1
  GPU 7 ->  node 1
[HCTR][08:41:29.902][WARNING][RK0][main]: Peer-to-peer access cannot be fully enabled.
[HCTR][08:41:29.903][INFO][RK0][main]: Start all2all warmup
[HCTR][08:41:30.083][INFO][RK0][main]: End all2all warmup
[HCTR][08:41:30.095][INFO][RK0][main]: Using All-reduce algorithm: NCCL
[HCTR][08:41:30.097][INFO][RK0][main]: Device 0: Tesla V100-SXM2-16GB
[HCTR][08:41:30.097][INFO][RK0][main]: Device 1: Tesla V100-SXM2-16GB
[HCTR][08:41:30.098][INFO][RK0][main]: Device 2: Tesla V100-SXM2-1

[HCTR][08:42:16.322][INFO][RK0][main]: Iter: 100 Time(100 iters): 8.19237s Loss: 0.140113 lr:0.168333
[HCTR][08:42:18.453][DEBUG][RK0][tid #140394662721280]: file_name_ deepfm_data_nvt/val/0.35ab81b16b4a409ba42a1baf89dcba52.parquet file_total_rows_ 571942
[HCTR][08:42:18.491][DEBUG][RK0][tid #140394654328576]: file_name_ deepfm_data_nvt/val/1.01854d707a564342aef3af44b814de1c.parquet file_total_rows_ 573919
[HCTR][08:42:18.534][DEBUG][RK0][tid #140394645935872]: file_name_ deepfm_data_nvt/val/2.7d7593c16af64625973ed246f68af624.parquet file_total_rows_ 572137
[HCTR][08:42:18.572][DEBUG][RK0][tid #140394528503552]: file_name_ deepfm_data_nvt/val/3.eec657484d40418cbf2648541592d09e.parquet file_total_rows_ 572545
[HCTR][08:42:18.610][DEBUG][RK0][tid #140394520110848]: file_name_ deepfm_data_nvt/val/4.e60c2f9421d84490bbc4de5f15ec5a0f.parquet file_total_rows_ 573664
[HCTR][08:42:18.651][DEBUG][RK0][tid #140394511718144]: file_name_ deepfm_data_nvt/val/5.883be83fecd74c1fbac00321911f2787.parque

[HCTR][08:43:05.354][DEBUG][RK0][tid #140394788546304]: file_name_ deepfm_data_nvt/train/6.92133f3ee3664684854969202958122f.parquet file_total_rows_ 4581782
[HCTR][08:43:06.072][DEBUG][RK0][tid #140394780153600]: file_name_ deepfm_data_nvt/train/7.9345ade3421b40a5803f518c48ae436f.parquet file_total_rows_ 4589169
[HCTR][08:43:09.539][INFO][RK0][main]: Iter: 600 Time(100 iters): 10.5255s Loss: 0.140006 lr:0.5
[HCTR][08:43:11.577][DEBUG][RK0][tid #140394662721280]: file_name_ deepfm_data_nvt/val/0.35ab81b16b4a409ba42a1baf89dcba52.parquet file_total_rows_ 571942
[HCTR][08:43:11.615][DEBUG][RK0][tid #140394654328576]: file_name_ deepfm_data_nvt/val/1.01854d707a564342aef3af44b814de1c.parquet file_total_rows_ 573919
[HCTR][08:43:11.653][DEBUG][RK0][tid #140394645935872]: file_name_ deepfm_data_nvt/val/2.7d7593c16af64625973ed246f68af624.parquet file_total_rows_ 572137
[HCTR][08:43:11.690][DEBUG][RK0][tid #140394528503552]: file_name_ deepfm_data_nvt/val/3.eec657484d40418cbf2648541592d09e.parqu

### ETPS: Distributed 
We want to distributed all tables across all gpus(the same way we use in  `hugectr.DistributedHashEmbedding`).

In [6]:
def generate_distributed_plan(table_size_array, num_gpus, plan_file):
    # place table across all gpus
    plan = []
    for gpu_id in range(num_gpus):
        distributed_plan = {
          'local_embedding_list': [table_id for table_id in range(len(table_size_array))],
          'table_placement_strategy': 'mp',
          'shard_id': gpu_id,
          'shards_count': num_gpus
        }
        plan.append([distributed_plan])

    # generate global view of table placement
    distributed_global_embedding_list = []
    for single_gpu_plan in plan:
        distributed_global_embedding_list.append(single_gpu_plan[0]['local_embedding_list'])
    for single_gpu_plan in plan:
        single_gpu_plan[0]['global_embedding_list'] = distributed_global_embedding_list
    print_plan(plan)
    # dump plan file
    import json
    with open(plan_file, 'w') as f:
        json.dump(plan, f, indent=4)

In [7]:
table_size_array = [203931, 18598, 14092, 7012, 18977, 4, 6385, 1245, 49,
                    186213, 71328, 67288, 11, 2168, 7338, 61, 4, 932, 15,
                    204515, 141526, 199433, 60919, 9137, 71, 34]
generate_distributed_plan(table_size_array, 8, "./distributed_plan.json")

single_gpu_plan index = 0
	local_embedding_list:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
	table_placement_strategy:mp
	shard_id:0
	shards_count:8
	global_embedding_list:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
	                     :[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
	                     :[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
	                     :[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
	                     :[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
	                     :[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
	                     :[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19

In [8]:
!python3 dlrm_train.py ./distributed_plan.json

HugeCTR Version: 3.7
====================================================Model Init=====================================================
[HCTR][08:44:05.384][WARNING][RK0][main]: The model name is not specified when creating the solver.
[HCTR][08:44:05.384][INFO][RK0][main]: Global seed is 1510630763
[HCTR][08:44:05.843][INFO][RK0][main]: Device to NUMA mapping:
  GPU 0 ->  node 0
  GPU 1 ->  node 0
  GPU 2 ->  node 0
  GPU 3 ->  node 0
  GPU 4 ->  node 1
  GPU 5 ->  node 1
  GPU 6 ->  node 1
  GPU 7 ->  node 1
[HCTR][08:44:17.340][WARNING][RK0][main]: Peer-to-peer access cannot be fully enabled.
[HCTR][08:44:17.341][INFO][RK0][main]: Start all2all warmup
[HCTR][08:44:17.532][INFO][RK0][main]: End all2all warmup
[HCTR][08:44:17.544][INFO][RK0][main]: Using All-reduce algorithm: NCCL
[HCTR][08:44:17.545][INFO][RK0][main]: Device 0: Tesla V100-SXM2-16GB
[HCTR][08:44:17.546][INFO][RK0][main]: Device 1: Tesla V100-SXM2-16GB
[HCTR][08:44:17.547][INFO][RK0][main]: Device 2: Tesla V100-SXM2-1

[HCTR][08:45:11.057][INFO][RK0][main]: Iter: 100 Time(100 iters): 15.3926s Loss: 0.144649 lr:0.168333
[HCTR][08:45:14.328][DEBUG][RK0][tid #139614102738688]: file_name_ deepfm_data_nvt/val/0.35ab81b16b4a409ba42a1baf89dcba52.parquet file_total_rows_ 571942
[HCTR][08:45:14.386][DEBUG][RK0][tid #139609623230208]: file_name_ deepfm_data_nvt/val/1.01854d707a564342aef3af44b814de1c.parquet file_total_rows_ 573919
[HCTR][08:45:14.444][DEBUG][RK0][tid #139609614837504]: file_name_ deepfm_data_nvt/val/2.7d7593c16af64625973ed246f68af624.parquet file_total_rows_ 572137
[HCTR][08:45:14.503][DEBUG][RK0][tid #139609606444800]: file_name_ deepfm_data_nvt/val/3.eec657484d40418cbf2648541592d09e.parquet file_total_rows_ 572545
[HCTR][08:45:14.562][DEBUG][RK0][tid #139609220577024]: file_name_ deepfm_data_nvt/val/4.e60c2f9421d84490bbc4de5f15ec5a0f.parquet file_total_rows_ 573664
[HCTR][08:45:14.620][DEBUG][RK0][tid #139609212184320]: file_name_ deepfm_data_nvt/val/5.883be83fecd74c1fbac00321911f2787.parque

[HCTR][08:46:40.586][DEBUG][RK0][tid #139614119524096]: file_name_ deepfm_data_nvt/train/6.92133f3ee3664684854969202958122f.parquet file_total_rows_ 4581782
[HCTR][08:46:41.968][DEBUG][RK0][tid #139614111131392]: file_name_ deepfm_data_nvt/train/7.9345ade3421b40a5803f518c48ae436f.parquet file_total_rows_ 4589169
[HCTR][08:46:48.555][INFO][RK0][main]: Iter: 600 Time(100 iters): 19.4819s Loss: 0.134819 lr:0.5
[HCTR][08:46:51.959][DEBUG][RK0][tid #139614102738688]: file_name_ deepfm_data_nvt/val/0.35ab81b16b4a409ba42a1baf89dcba52.parquet file_total_rows_ 571942
[HCTR][08:46:52.017][DEBUG][RK0][tid #139609623230208]: file_name_ deepfm_data_nvt/val/1.01854d707a564342aef3af44b814de1c.parquet file_total_rows_ 573919
[HCTR][08:46:52.075][DEBUG][RK0][tid #139609614837504]: file_name_ deepfm_data_nvt/val/2.7d7593c16af64625973ed246f68af624.parquet file_total_rows_ 572137
[HCTR][08:46:52.134][DEBUG][RK0][tid #139609606444800]: file_name_ deepfm_data_nvt/val/3.eec657484d40418cbf2648541592d09e.parqu

### Compare performance between different ETPS
We can see the iteration time for dataparallel + localized is 103.45s while for distributed is 190.85s, which means different ETPS can greatly affect the performance of embedding. So it's better to put embedding table as data parallel or localized when the table can be fitted into single GPU.